In [67]:
#To check for precedents
import json
with open('jsonformatter.txt', 'r') as f:
    with open("Precedents output.txt", 'w') as out:
        data = json.load(f)

        # print(type(data['label']))
        for case in data:
            out.write('Case id : ' + str(case['id']) + '\n')
            annotation = case['annotations'][0]
            # print(annotation['result'])
            for key in annotation['result']:
                out.write(key['value']['labels'][0] + '\n')

            out.write("\n")

In [68]:
#To check for precedents relied
import json
with open('jsonformatter.txt', 'r') as f:
    with open("Precedents relied output.txt", 'w') as out:
        data = json.load(f)

        for case in data:
            out.write('Case id : ' + str(case['id']) + '\n')
            annotation = case['annotations'][0]
            for key in annotation['result']:
                if key['value']['labels'][0] == 'PRE_RELIED':
                    out.write('PRE_RELIED : ' + key['value']['text'] + '\n\n')

            out.write("\n")